In [2]:
import numpy as np
import time
import mujoco
import mujoco.viewer

In [3]:
modelfilename = './test4n.xml'#'./model/mjcf/dora2_stand_fix.xml'
m = mujoco.MjModel.from_xml_path(modelfilename)
d = mujoco.MjData(m)
mujoco.mj_forward(m, d)

In [36]:
modelfilename = './test4n.xml'#'./model/mjcf/dora2_stand_fix.xml'
m = mujoco.MjModel.from_xml_path(modelfilename)
d = mujoco.MjData(m)
mujoco.mj_forward(m, d)
mrec = [[] for _ in range(45)]
lastqdot = d.qvel.copy()
lastM = np.zeros((m.nv, m.nv))
mujoco.mj_fullM(m, lastM, d.qM)
lastc = d.qfrc_bias.copy()
lasttime = d.time
B = np.zeros((m.nv, m.nu))
B[6:,:] = np.identity(m.nu)
# print(B)
# print(d.ctrl)
etaue = np.zeros(m.nv)
expect_vel = np.zeros(m.nv)

def mycontroller(m,d):
    global lastqdot
    global lastM
    global lastc
    global lasttime
    global etaue
    global expect_vel

    mujoco.mj_forward(m, d)
    M = np.zeros((m.nv, m.nv))
    mujoco.mj_fullM(m, M, d.qM)
    Mcf = M[:,:6]
    Mcj = M[:,6:]
    MBM = np.block([Mcf,-B])
    c = d.qfrc_bias
    if d.time < 0.0001:
       taue = np.zeros(m.nv)
    else:
        # taue = 0.5*(lastM + M)@(d.qvel - lastqdot)/(d.time-lasttime) + (lastc+c)/2 - B@d.ctrl
        # taue = M@(d.qvel - lastqdot)/(d.time-lasttime) + c - B@d.ctrl
        taue = lastM@(d.qvel - lastqdot)/(d.time-lasttime) + lastc - B@d.ctrl
        etaue -= 0.8*lastM@(expect_vel - d.qvel)/(d.time-lasttime)
    qa = np.zeros(m.nu)

    kp = 100
    kd = 10
    kp2 = kp
    kd2 = kd
    kp3 = 5
    kd3 = 3
    theta = 0
    initald = 10
    #参数说明：关节名称    Kp  位置目标  Kd     速度目标 D
    cl = {'HIPRY'  :[   kp,     0,  kd,     0,  initald],
          'HIPRX'  :[   kp, theta,  kd,     0,  initald],
          'KNEERY' :[   kp,     0,  kd,     0,  initald],
          'HIPRZ'  :[   kp,     0,  kd,     0,  initald],
          'ANKLERY':[   kp2,    0,  kd2,    0,  initald],
          'ANKLERX':[   kp3,    0,  kd3,    0,  initald],
          'HIPLY'  :[   kp,     0,  kd,     0,  initald],
          'HIPLX'  :[   kp, theta,  kd,     0,  initald],
          'HIPLZ'  :[   kp,     0,  kd,     0,  initald],
          'KNEELY' :[   kp,     0,  kd,     0,  initald],
          'ANKLELY':[   kp2,    0,  kd2,    0,  initald],
          'ANKLELX':[   kp3,    0,  kd3,    0,  initald],
          'WAISTY' :[   kp,     0,  kd,     0,  initald],
          'WAISTX' :[   kp,     0,  kd,     0,  initald]}
    # cl = {'HIPRY'  :[   kp,     -0.5,  kd,     0,  initald],
    #       'HIPRX'  :[   kp, theta,  kd,     0,  initald],
    #       'KNEERY' :[   kp,     1,  kd,     0,  initald],
    #       'HIPRZ'  :[   kp,     0,  kd,     0,  initald],
    #       'ANKLERY':[   kp2,    -0.5,  kd2,    0,  initald],
    #       'ANKLERX':[   kp3,    0,  kd3,    0,  initald],
    #       'HIPLY'  :[   kp,     -0.5,  kd,     0,  initald],
    #       'HIPLX'  :[   kp, theta,  kd,     0,  initald],
    #       'HIPLZ'  :[   kp,     0,  kd,     0,  initald],
    #       'KNEELY' :[   kp,     1,  kd,     0,  initald],
    #       'ANKLELY':[   kp2,    -0.5,  kd2,    0,  initald],
    #       'ANKLELX':[   kp3,    0,  kd3,    0,  initald],
    #       'WAISTY' :[   kp,     0,  kd,     0,  initald],
    #       'WAISTX' :[   kp,     0,  kd,     0,  initald]}
    for jn,tm in cl.items():
        m.dof_armature[m.jnt(jn).id+5] = 0.01
        qa[m.jnt(jn).id-1] = (-tm[0]*(d.qpos[m.jnt(jn).id+6]-tm[1]) -tm[2]*(d.qvel[m.jnt(jn).id+5]-tm[3]))
        expect_vel[m.jnt(jn).id+5] = d.qvel[m.jnt(jn).id+5] + qa[m.jnt(jn).id-1]*(d.time-lasttime)
    # qddftau = (np.linalg.inv(MBM)@(0.78*taue - Mcj@qa - c))
    qddftau = (np.linalg.inv(MBM)@(etaue - Mcj@qa - c))
    d.ctrl = qddftau[6:]
    expect_vel[:6] = d.qvel[:6] + qddftau[:6]*(d.time-lasttime)
    # lastqdot
    # lastM
    # lastc
    # lasttime
    lastqdot = d.qvel.copy()
    lastM = M.copy()
    lastc = c.copy()
    lasttime = d.time
    
    # print(taue)


def set_cam_viewer(m, viewer):
    # 设置镜头跟随
    viewer.cam.type = mujoco.mjtCamera.mjCAMERA_TRACKING
    viewer.cam.trackbodyid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_BODY, "link0")
    viewer.cam.distance = 3.0# 设置摄像机距离 
    viewer.cam.elevation = -20# 设置摄像机仰角
    
    # 设置摄像机为固定点模式
    # viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
    # 设定摄像机的位置和朝向
    # viewer.cam.lookat = [0, 0, 0]  # 摄像机视线的目标点
    # viewer.cam.distance = 5.0      # 摄像机与目标点的距离
    # viewer.cam.elevation = -20     # 摄像机仰角
    # viewer.cam.azimuth = 45        # 摄像机方位角

with mujoco.viewer.launch_passive(m, d) as viewer:
  # set_cam_viewer(m, viewer)
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  ii = 0
#   d.qpos[2] = 1.5
#   d.qpos[7] = -0.5
#   d.qpos[10] = 1
#   d.qpos[11] = -0.5
#   d.qpos[13] = -0.5
#   d.qpos[16] = 1
#   d.qpos[17] = -0.5
#   qst = pin.Quaternion(pin.AngleAxis(0,np.array([0,0,1])).matrix()).coeffs().copy()
#   qslt = qst[3]
#   qst[3] = qst[2]
#   qst[2] = qst[1]
#   qst[1] = qst[0]
#   qst[0] = qslt
#   d.qpos[3:7] = qst
  mujoco.mj_forward(m, d)
  while viewer.is_running() and time.time() - start < 25:
    step_start = time.time()

    if ii % 2 == 0:
        mycontroller(m,d)
    ii += 1
    # print(d.ctrl)

    mrec[24].append(d.ctrl[0])
    mrec[25].append(d.ctrl[3])
    mrec[26].append(d.ctrl[4])
    
    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics. 
    mujoco.mj_step(m, d)
    mrec[1].append(d.subtree_com[1][0])
    mrec[2].append(d.subtree_com[1][2])
    mrec[3].append(d.subtree_com[1][1])
    mrec[4].append(0.1*np.sin(d.time - 2.5)) #这里也要跟着改
    angmomm = np.zeros((3,m.nv))
    mujoco.mj_angmomMat(m,d,angmomm,0)
    hhere = angmomm@d.qvel
    mrec[5].append(hhere[0])
    mrec[6].append(hhere[1])
    mrec[7].append(hhere[2])
    # mrecord8.append(d.qpos[8])
    # mrecord9.append(d.qpos[14])
    mujoco.mj_subtreeVel(m,d)
    mrec[30].append(d.subtree_linvel[1][0])
    mrec[31].append(d.subtree_linvel[1][1])
    mrec[32].append(d.subtree_linvel[1][2])

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

: 